# 

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
test_dir = '/kaggle/input/raf-db-dataset/DATASET/test'
train_dir = '/kaggle/input/raf-db-dataset/DATASET/train'

In [3]:
!ls /kaggle/input/raf-db-dataset/DATASET

test  train


In [4]:
from PIL import Image,ImageOps
def dataset_by_path(path):

    X = []
    y = []
    required_size=(32, 32)
    i=0

    for class_name in os.listdir(path):
        path_to =path+'/'+class_name
        for img_path in os.listdir(path_to):
            y.append(int(class_name))
            
            image = ImageOps.grayscale(Image.open(path_to+'/'+img_path)).resize(required_size)
            img_arr = np.array(image).reshape(32*32)
            X.append(img_arr)
    return np.array(X),np.array(y)



In [5]:
test_f ,test_label =dataset_by_path(test_dir)
train_f ,train_label =dataset_by_path(train_dir)


In [6]:
print(train_f.shape)
print(train_label.shape)

(12271, 1024)
(12271,)


In [7]:
from pandas import DataFrame
import warnings
warnings.filterwarnings("ignore")

In [8]:
name_col= [str(i) for i in range(1024)]
X_train = DataFrame()
X_train[name_col] = train_f
Y_train = DataFrame()
Y_train['label'] = train_label
X_test = DataFrame()
X_test[name_col] = test_f
Y_test = DataFrame()
Y_test['label'] = test_label

/tmp/ipykernel_32/4050256713.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[name_col] = train_f
/tmp/ipykernel_32/4050256713.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[name_col] = train_f
/tmp/ipykernel_32/4050256713.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_

In [9]:
Y_train.value_counts()

label
4        4772
7        2524
5        1982
1        1290
3         717
6         705
2         281
Name: count, dtype: int64

In [17]:
import optuna

import catboost as cb
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

def objective(trial):
    x, xt, y, yt = train_f,test_f,train_label,test_label
    param = {
        "objective": trial.suggest_categorical("objective", ["MultiClass", "MultiClassOneVsAll"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        ),
        "used_ram_limit": "25gb",
        'max_depth': trial.suggest_int('max_depth', 3, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 50, 600),
        'max_bin': trial.suggest_int('max_bin', 100, 400),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log=True),
        'random_seed': 42,
        'task_type': 'CPU',
        'eval_metric': 'Accuracy',
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        
    pca_param ={
        "n_components": trial.suggest_float("n_components", 0.5, 0.9999),
        'power_iteration_normalizer': trial.suggest_categorical('power_iteration_normalizer',
            ['auto', 'QR', 'LU', 'none']
        ),
        'svd_solver':'full',

    }
    
    pca = PCA(**pca_param).fit(x,y)
    x = pca.transform(x)
    xt = pca.transform(xt)
    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(x, y, eval_set=[(xt, yt)], verbose=0, early_stopping_rounds=100)
    preds = gbm.predict(xt)
    pred_labels = np.rint(preds)

    ac = gbm.get_best_score()['validation']['Accuracy']
    print(ac)
    return ac

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-10-21 18:44:57,882] A new study created in memory with name: no-name-2abd76d1-59b6-492a-afd7-8a025229b97f
[I 2023-10-21 18:45:15,481] Trial 1 finished with value: 0.386245110821382 and parameters: {'objective': 'MultiClassOneVsAll', 'colsample_bylevel': 0.039277298080348064, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'max_depth': 4, 'learning_rate': 0.02, 'n_estimators': 562, 'max_bin': 170, 'min_data_in_leaf': 243, 'l2_leaf_reg': 0.10065062951542429, 'subsample': 0.9492627909621454, 'n_components': 0.6323972209024556, 'power_iteration_normalizer': 'none'}. Best is trial 1 with value: 0.386245110821382.


0.386245110821382


[I 2023-10-21 18:45:48,265] Trial 0 finished with value: 0.40123859191655803 and parameters: {'objective': 'MultiClass', 'colsample_bylevel': 0.06414606216695551, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'max_depth': 10, 'learning_rate': 0.005, 'n_estimators': 568, 'max_bin': 103, 'min_data_in_leaf': 2, 'l2_leaf_reg': 0.0006040891759402597, 'bagging_temperature': 6.460121966758063, 'n_components': 0.8592869110449686, 'power_iteration_normalizer': 'LU'}. Best is trial 0 with value: 0.40123859191655803.


0.40123859191655803


[I 2023-10-21 18:46:06,719] Trial 5 finished with value: 0.386245110821382 and parameters: {'objective': 'MultiClass', 'colsample_bylevel': 0.0643075701013641, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'max_depth': 4, 'learning_rate': 0.08, 'n_estimators': 157, 'max_bin': 307, 'min_data_in_leaf': 122, 'l2_leaf_reg': 0.00022463455150429314, 'subsample': 0.5839727668828589, 'n_components': 0.6155431433667226, 'power_iteration_normalizer': 'LU'}. Best is trial 0 with value: 0.40123859191655803.


...

0.4602346805736636


[I 2023-10-21 18:53:48,171] Trial 16 finished with value: 0.386245110821382 and parameters: {'objective': 'MultiClass', 'colsample_bylevel': 0.05051635155336595, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'max_depth': 3, 'learning_rate': 0.005, 'n_estimators': 350, 'max_bin': 317, 'min_data_in_leaf': 218, 'l2_leaf_reg': 0.03978176987713919, 'subsample': 0.7541949856909858, 'n_components': 0.9879278625950245, 'power_iteration_normalizer': 'LU'}. Best is trial 9 with value: 0.5736636245110821.


0.386245110821382


[I 2023-10-21 18:54:37,925] Trial 17 finished with value: 0.5104302477183833 and parameters: {'objective': 'MultiClassOneVsAll', 'colsample_bylevel': 0.028842338335693794, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'max_depth': 13, 'learning_rate': 0.1, 'n_estimators': 445, 'max_bin': 251, 'min_data_in_leaf': 278, 'l2_leaf_reg': 0.07333650259969578, 'subsample': 0.772143771869676, 'n_components': 0.8316492044312376, 'power_iteration_normalizer': 'LU'}. Best is trial 9 with value: 0.5736636245110821.


0.5104302477183833


In [16]:
x, xt, y, yt = train_f,test_f,train_label,test_label

gbm = cb.CatBoostClassifier(iterations=10,       eval_metric='Accuracy').fit(x, y, eval_set=[(xt, yt)], verbose=0, early_stopping_rounds=100)
print(gbm.get_best_score())


{'learn': {'Accuracy': 0.5582267133892919, 'MultiClass': 1.216754045272294}, 'validation': {'Accuracy': 0.5365058670143416, 'MultiClass': 1.2801059650426916}}
